## Install packages

In [ ]:
! pip install ../input/mmdetectionv260/addict-2.4.0-py3-none-any.whl
! pip install ../input/mmdetectionv260/mmcv_full-latesttorch1.6.0cu102-cp37-cp37m-manylinux1_x86_64.whl
! pip install ../input/mmdetectionv260/mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl
! pip install ../input/mmdetection-package/mmdet-2.7.0-py3-none-any.whl

In [ ]:
! pip install ../input/mmdetection-package/torch-1.6.0-cp37-cp37m-linux_x86_64.whl

In [ ]:
#! pip install ../input/mmdetection-package/albumentations-0.5.2-py3-none-any.whl

In [ ]:
! cp -r ../input/mmdetection-package/mmdetection/mmdetection/ ./mmdetection

In [ ]:
! cp -r ../input/mmdetection-wheat-models/attention_stage1/faster_rcnn_r50_fpn_attention_0010_dcn_albu_1x4_1x_bWheat_kaggle.py ./config.py
! cp -r ../input/mmdetection-wheat-models/attention_stage1/epoch_12.pth ./model.pth

In [ ]:
CONFIG_FILE = './config.py'
CHECKPOINT_PATH = './model.pth'
TEST_IMG_DIR = '../input/global-wheat-detection/test'

## Import packages

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import cv2

from tqdm import tqdm

import torch
import mmcv
from mmdet.apis import init_detector, inference_detector

## Pseudo labeling

### Create test annotations

In [ ]:
TEST_ANN_FILE = './annotation_test.json'

anns = []
for img_name in tqdm(os.listdir(TEST_IMG_DIR)):
    if not img_name.endswith('.jpg'):
        continue
    anns.append(dict(filename=img_name, boxes=[]))

In [ ]:
images = []
annotations = []
obj_count = 0
for idx, v in tqdm(enumerate(anns)):
    filename = v['filename']
    img_path = os.path.join(TEST_IMG_DIR, filename)
    height, width = mmcv.imread(img_path).shape[:2]

    images.append(dict(
        id=idx,
        file_name=filename,
        height=height,
        width=width))

    for box in v['boxes']:
        data_anno = dict(
            image_id=idx,
            id=obj_count,
            category_id=int(box['label']),
            bbox=[box['left'], box['top'], box['width'], box['height']],
            area=box['width'] * box['height'],
            iscrowd=0)

        annotations.append(data_anno)
        obj_count += 1

coco_format_json = dict(
    images=images,
    annotations=annotations,
    categories=[dict(id=0, name='wheat_head')])
mmcv.dump(coco_format_json, TEST_ANN_FILE)

### Modify test config

In [ ]:
test_config = './test_config.py'
cfg = mmcv.Config.fromfile(CONFIG_FILE)
cfg.data.test.ann_file = TEST_ANN_FILE

with open(test_config, 'w') as f:
    f.write(cfg.pretty_text)

In [ ]:
! python ./mmdetection/tools/test.py {test_config} {CHECKPOINT_PATH} --format-only --eval-options jsonfile_prefix=./result

In [ ]:
with open('./result.bbox.json', 'r') as f1, open(TEST_ANN_FILE, 'r') as f2:
    result_info = json.load(f1)
    annotations_info = json.load(f2)
    
    for i, ann in tqdm(enumerate(result_info)):
        if ann['score'] < 0.5:
            continue
        annotation = ann
        annotation['id'] = i
        annotation['area'] = ann['bbox'][2] * ann['bbox'][3]
        annotation['iscrowd'] = 0
        annotations_info['annotations'].append(annotation)
    
    mmcv.dump(annotations_info, './annotation_new.json')

### Train with pseudo label

In [ ]:
train_config = './train_config.py'
cfg = mmcv.Config.fromfile(CONFIG_FILE)
cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4
cfg.data.train.ann_file = './annotation_new.json'
cfg.data.train.img_prefix = TEST_IMG_DIR
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Albu',
        transforms=cfg.albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.3,
            filter_lost_elements=True),
        keymap=dict(
            img='image',
            gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Resize', img_scale=(1024, 1024), keep_ratio=True), # Change scale
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
cfg.lr_config = dict(
    policy='fixed',
    warmup=None
)
cfg.total_epochs = 2
cfg.model.pretrained = None
cfg.load_from = CHECKPOINT_PATH
cfg.log_config = dict(
    interval=1,
    hooks=[
        dict(type='TextLoggerHook')
    ])

with open(train_config, 'w') as f:
    f.write(cfg.pretty_text)

In [ ]:
! python ./mmdetection/tools/train.py {train_config} --no-validate --work-dir ./pseudo

In [ ]:
! ls ./pseudo

## Make prediction

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
config = mmcv.Config.fromfile(CONFIG_FILE)
config.model.pretrained = None
config.data.test.pipeline[1]['img_scale'] = [(1280, 1280), (1408, 1408)]

model = init_detector(config, './pseudo/epoch_1.pth', device=device)
model.eval()

In [ ]:
results = []
score_threshold = 0.3
with torch.no_grad():
    for img_name in tqdm(os.listdir(TEST_IMG_DIR)):
        img_pth = os.path.join(TEST_IMG_DIR, img_name)
        image = mmcv.imread(img_pth)
        
        # Predict with original image
        result = inference_detector(model, image)
        boxes = result[0][:, :4]
        scores = result[0][:, 4]
        if len(boxes) > 0:
            boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        # Remove prediction with score under threshold
        boxes = boxes[scores >= score_threshold]
        scores = scores[scores >= score_threshold]

        result = {
            'image_id': img_name[:-4],
            'PredictionString': format_prediction_string(boxes, scores)
        }

        results.append(result)

In [ ]:
import matplotlib.pyplot as plt

size = 300
idx =-1
font = cv2.FONT_HERSHEY_SIMPLEX 
image = cv2.imread(img_pth, cv2.IMREAD_COLOR)
fontScale = 1
color = (255, 0, 0)

thickness = 2
for b, s in zip(boxes, scores):
    b = [int(a) for a in b]
    image = cv2.rectangle(image, (b[0],b[1]), (b[0]+b[2],b[1]+b[3]), (255,0,0), 1) 
    image = cv2.putText(image, '{:.2}'.format(s), (b[0]+np.random.randint(20),b[1]), font,  
                   fontScale, color, thickness, cv2.LINE_AA)
plt.figure(figsize=[20,20])
plt.imshow(image[:,:,::-1])
plt.show()

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
test_df.head()